In [1]:
import numpy as np
import pandas as pd

In [28]:
orders = pd.read_csv('../data/orders.csv')
train_orders = pd.read_csv('../data/order_products__train.csv')
train_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [30]:
train = orders[orders['eval_set'] == 'train']
prior = orders[orders['eval_set'] == 'prior']
test = orders[orders['eval_set'] == 'test']
train.shape, prior.shape, test.shape

((131209, 7), (3214874, 7), (75000, 7))

In [33]:
len(train['order_id'].unique()), len(prior['order_id'].unique()), len(test['order_id'].unique())

(131209, 3214874, 75000)

In [35]:
len(orders['order_id'].unique()), orders.shape

(3421083, (3421083, 7))

In [3]:
len(train_orders['order_id'].unique()), len(train_orders['order_id'].unique()) / train_orders.shape[0]

(131209, 0)

In [4]:
temp = train_orders[train_orders['order_id'] == 1]
temp['rank'] = \
    list(reversed(train_orders[train_orders['order_id'] == 1]['add_to_cart_order'].tolist()))
temp

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,order_id,product_id,add_to_cart_order,reordered,rank
0,1,49302,1,1,8
1,1,11109,2,1,7
2,1,10246,3,0,6
3,1,49683,4,0,5
4,1,43633,5,1,4
5,1,13176,6,0,3
6,1,47209,7,0,2
7,1,22035,8,1,1


In [5]:
train_orders['rank'] = 1
train_orders.head(), train_orders.tail()

(   order_id  product_id  add_to_cart_order  reordered  rank
 0         1       49302                  1          1     1
 1         1       11109                  2          1     1
 2         1       10246                  3          0     1
 3         1       49683                  4          0     1
 4         1       43633                  5          1     1,
          order_id  product_id  add_to_cart_order  reordered  rank
 1384612   3421063       14233                  3          1     1
 1384613   3421063       35548                  4          1     1
 1384614   3421070       35951                  1          1     1
 1384615   3421070       16953                  2          1     1
 1384616   3421070        4724                  3          1     1)

In [22]:
def add_rank(df, order_id):
    df.loc[df['order_id'] == order_id, 'rank'] = \
        list(reversed(df[df['order_id'] == order_id]['add_to_cart_order'].tolist()))
#     print(df[df['order_id'] == order_id].head(3))
#     print(df[df['order_id'] == order_id].tail(3))

In [26]:
# %%time

import multiprocessing
pool = multiprocessing.Pool()

num = len(train_orders['order_id'].unique())
map(add_rank, [train_orders]*num, train_orders['order_id'].unique())

In [3]:
test = orders[orders['eval_set'] == 'test']
test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [10]:
print(len(test['user_id'].unique()))
test.shape

75000


(75000, 7)

In [17]:
print(len(train['user_id'].unique()))
print(len(prior['user_id'].unique()))
print(len(prior['user_id'].unique()) / prior.shape[0] * 100, '%')

131209
206209
6.414217166831422 %


In [19]:
user_ids = test['user_id'].values
new_orders = orders[orders['user_id'].isin(user_ids)]
new_train = new_orders[new_orders['eval_set'] == 'train']
new_prior = new_orders[new_orders['eval_set'] == 'prior']
new_train.shape, new_prior.shape

((0, 7), (1167497, 7))

In [20]:
len(new_prior['user_id'].unique())

75000